In [31]:
from scipy.misc import imread
from glob import glob
import pandas as pd
import numpy as np

In [39]:
DST_FP  = [ './data/pv_panels/training/training_scheme.csv',
            './data/pv_panels/validation/validation_scheme.csv' ]

filepaths = [  { 'images' : glob('./data/pv_panels/training/images/*.png'), # Training
                 'masks'  : glob('./data/pv_panels/training/masks/*.png') },
               { 'images' : glob('./data/pv_panels/validation/images/*.png'), # Validation
                 'masks'  : glob('./data/pv_panels/validation/masks/*.png')} ]

for i in [0, 1]:
    image_names = [fp.split('/')[-1] for fp in filepaths[i]['images']]
    mask_names  = [fp.split('/')[-1] for fp in filepaths[i]['masks']]

    lists_match     = np.prod([image_names[j] == mask_names[j] for j in range(len(image_names))]) # Check the lists are aligned
    
    image_classes   = [np.sum(imread(fp, mode = 'L')) > 1 for fp in filepaths[i]['masks']]
    imbalanced_data = pd.DataFrame({
                        'name'  : image_names,
                        'class' : image_classes
                    })
    
    if (i == 0): # Balance the training data
            class_counts    = imbalanced_data['class'].value_counts()
            class_imbalance = class_counts[False] - class_counts[True]

            sample          = imbalanced_data[imbalanced_data['class']].sample(n = class_imbalance, replace = True)
            balanced_data   = imbalanced_data.append(sample)
            writable_data   = balanced_data.sample(n = balanced_data.shape[0], replace = False) # shuffle the data
            
    if (i == 1): # Do not balance the validation data, but shuffle it
            writable_data = imbalanced_data.sample(n = imbalanced_data.shape[0], replace = False)
    
    writable_data.to_csv(DST_FP[i], index = False)
        